In [4]:
import cv2
from PIL import Image
import glob, os
import json

In [5]:
def norm2abs(bbx_coords, image_shape):
    x, y, w, h, = bbx_coords

    # get x and y
    x_reg = int(x * image_shape[1])
    y_reg = int(y * image_shape[0])
    w_reg = int(w * image_shape[1])
    h_reg = int(h * image_shape[0])

    bbx_coords_reg = [x_reg, y_reg, w_reg, h_reg]
    return bbx_coords_reg

In [6]:
# Check image dimensions, bounding box coordinates and labels
# We will compare the normalised and absolute coordinates, we assume the two json files are keyed identically
root = 'C:/Users/drago/Documents/gitrepos/recycle_robot/Autogathered_Dataset/1/'
directory = root + 'data/'
check_shape = False

abs_labels_dir = root + 'Completed_Labels.json'

# Open the json file containing the absolute labels
with open(abs_labels_dir) as abs_labels_json:
    abs_labels_dict = json.load(abs_labels_json)

#last_annotation_index = 0
# Iterate through each image in abs_labels, extracting filename and id
for file in [{'name': image['file_name'].split('/')[1], 'id': image['id']} for image in abs_labels_dict['images'][:]]:
    if file['name'].endswith('.png'):
        # Read in the image
        dataset_img = cv2.imread(directory + file['name'])
        
        # Extract the bounding boxes for each screw in the image
        label_ids = []
        #match_index = 0
        for index, abs_label in enumerate(abs_labels_dict['annotations'][:]):
            # Find the annotations that match the image
            # I want to know which images have annotations of area zero
            if abs_label['image_id'] == file['id']:

                # Read in absolute bounding box coordinates
                coordinates = abs_label['bbox']

                #Display image with bounding box
                start_point = (int(coordinates[0]), int(coordinates[1]))
                end_point = (int(coordinates[0] + coordinates[2]), int(coordinates[1] + coordinates[3]))
                
                print(start_point, end_point)
                print('\n')

                if abs_label['category_id'] == 2:
                    colour = (0, 0, 255)
                if abs_label['category_id'] == 4:
                    colour = (255, 0, 0)
                # draw the rectangle
                cv2.rectangle(dataset_img, start_point, end_point, color=colour, thickness= 1, lineType=cv2.LINE_8)

                print("Image details: {}".format(file))
                print("Label details: {}".format(abs_label))
                
                #match_index = index

        # Update the last annotation index
        #last_annotation_index = match_index

        print("Zero area bounding box found in image {}".format(file['name']))
        # display the output
        print("Dimensions {}: {}".format(file['name'], dataset_img.shape))
        cv2.imshow('imageRectangle', dataset_img)
        cv2.waitKey(0)
    else:
        continue

cv2.destroyAllWindows()

# # Check Github dataset iamge dimensions
# directory = 'C:/Users/drago/Documents/gitrepos/Cross-Recessed-Screw_Deep-Learning-Datasets/Training_Images/'
# for filename in os.listdir(directory):
#     if filename.endswith('.jpg'):
#         dataset_img = cv2.imread(directory + filename)
        
#         print("Dimensions {}: {}", (filename, dataset_img.shape))
#         continue
#     else:
#         continue

# directory = 'C:/Users/drago/Documents/gitrepos/Cross-Recessed-Screw_Deep-Learning-Datasets/Training_Images/'

(369, 365) (390, 388)


(279, 11) (298, 29)


(276, 9) (300, 31)


(367, 360) (400, 392)


(312, 584) (338, 610)


(308, 580) (347, 619)


(363, 288) (382, 307)


(360, 286) (384, 309)


(356, 525) (394, 564)


(387, 417) (410, 440)


(289, 64) (313, 86)


(386, 415) (419, 447)


(292, 66) (310, 85)


(0, 246) (7, 264)


(0, 242) (11, 268)


(325, 446) (344, 464)


(24, 204) (52, 232)


(305, 541) (320, 547)


(324, 442) (348, 465)


(301, 539) (326, 550)


(460, 223) (480, 244)


(456, 211) (480, 252)


(113, 465) (137, 489)


(105, 460) (142, 497)


(62, 25) (86, 49)


(56, 524) (77, 545)


(58, 21) (87, 50)


(298, 433) (320, 455)


(55, 48) (74, 68)


(461, 207) (480, 230)


(51, 43) (79, 71)


(459, 201) (480, 232)


(3, 611) (52, 640)


(182, 230) (203, 251)


(0, 568) (12, 593)


(428, 227) (449, 249)


(0, 567) (14, 596)


(55, 453) (83, 480)


(136, 114) (159, 136)


(284, 448) (302, 466)


(404, 209) (430, 236)


(147, 537) (166, 545)


(131, 105) (164, 137)


(281, 446) (306

In [ ]:
# Convert git dataset to 480x640 dimensions
size = 480, 640

for infile in glob.glob(directory + "*.jpg"):
    file, ext = os.path.splitext(infile)
    file_name = os.path.basename(file)
    with Image.open(infile) as im:
        im.thumbnail(size)
        im.save(directory + 'thumbnail/' + file_name + '.jpg', "JPEG")

In [ ]:

# Check those dimensions match all images within their respective dataset
# NOTE: There are some rotated 90 deg
# Now, think of some creative way to handle the difference between the image sizes,
#  the new cameras image size and the size of model needed to take these as inputs?
# Do I need to scale down the size of the images for prediction? How would I make use of the predictions?
#  Would scaling them actually work?